# Fine-Tune a Generative AI Model for Machine Translation task

In this notebook, I will fine-tune an existing LLM from Hugging Face for enhanced machine translation. I will use the [MarianMT](https://huggingface.co/docs/transformers/en/model_doc/marian) model, which provides a high quality instruction tuned model and can translation text out of the box. To improve the inferences, you will explore a full fine-tuning approach and evaluate the results with BLEU metrics. Then you will perform Parameter Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

# Table of Contents

- [ 1 - Load Required Dependencies, Dataset and LLM](#1)
  - [ 1.1 - Set up Required Dependencies](#1.1)
  - [ 1.2 - Load Dataset and LLM](#1.2)
  - [ 1.3 - Test the Model with Zero Shot Inferencing](#1.3)
- [ 2 - Perform Full Fine-Tuning](#2)
  - [ 2.1 - Preprocess the Machine Translation Dataset](#2.1)
  - [ 2.2 - Fine-Tune the Model with the Preprocessed Dataset](#2.2)
  - [ 2.3 - Evaluate the Model Qualitatively (Human Evaluation)](#2.3)
  - [ 2.4 - Evaluate the Model Quantitatively (with BLEU Metric)](#2.4)
- [ 3 - Perform Parameter Efficient Fine-Tuning (PEFT)](#3)
  - [ 3.1 - Setup the PEFT/LoRA model for Fine-Tuning](#3.1)
  - [ 3.2 - Train PEFT Adapter](#3.2)
  - [ 3.3 - Evaluate the Model Qualitatively (Human Evaluation)](#3.3)
  - [ 3.4 - Evaluate the Model Quantitatively (with BLEU Metric)](#3.4)

<a name='1'></a>
## 1 - Load Required Dependencies, Dataset and LLM

<a name='1.1'></a>
### 1.1 - Set up Required Dependencies

Install the required packages for the LLM and datasets.


In [2]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Ignored the following yanked versions: 0.3.0a0
ERROR: Could not find a version that satisfies the requirement torchdata==0.5.1 (from versions: 0.3.0a1, 0.3.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.8.0, 0.9.0, 0.10.0, 0.10.1)
ERROR: No matching distribution found for torchdata==0.5.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook. 

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<a name='1.2'></a>
### 1.2 - Load Dataset and LLM

You are going to continue experimenting with the [En-Az](https://huggingface.co/datasets/Zarifa/English-To-Azerbaijani) Hugging Face dataset. It contains 5,000+ sentences with the corresponding manually labeled translation. 

In [4]:
huggingface_dataset_name = "Zarifa/English-To-Azerbaijani"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 5161
    })
})

Load the pre-trained [MarianMT](https://huggingface.co/Helsinki-NLP/opus-mt-az-en) and its tokenizer directly from HuggingFace. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [5]:
model_name = "Helsinki-NLP/opus-mt-az-en"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


It is possible to pull out the number of model parameters and find out how many of them are trainable. The following function can be used to do that, at this stage, you do not need to go into details of it. 

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 56061952
all model parameters: 56586240
percentage of trainable model parameters: 99.07%


<a name='1.3'></a>
### 1.3 - Test the Model with Zero Shot Inferencing

Test the model with the zero shot inferencing. You can see that the model struggles to translate the test compared to the baseline result, but it does pull out some important information from the text which indicates the model can be fine-tuned to the task at hand.

In [10]:
index = 200

sentence = dataset['train'][index]['translation']['aze']
translate = dataset['train'][index]['translation']['en']

prompt = f"""
Translate the following sentence.

{sentence}

Translation:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN Translation:\n{translate}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Translate the following sentence.

Başçı dörd illiyinə seçildi.

Translation:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN Translation:
The president was elected for four years.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Technology was elected for four years:


<a name='2.1'></a>
### 2.1 - Preprocess the Machine Translation Dataset

You need to convert the sentence-translate (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Translate the following conversation` and to the start of the Translate with `Translate` as follows:

Training prompt (Translation):
```
Translate the following conversation.

    Başçı dörd illiyinə seçildi.
    
Translate: 
```

Training response (Translate):
```
The president was elected for four years.
```

Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 5161
    })
})

In [35]:
def tokenize_function(example):
    start_prompt = 'Translate the following conversation.\n\n'
    end_prompt = '\n\nTranslate: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in [ex['aze'] for ex in example['translation']]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer([ex['en'] for ex in example['translation']], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [36]:
tokenized_datasets = tokenized_datasets.remove_columns(['id'])

To save some time in the lab, I will subsample the dataset:

*Note* : I do not have validation so I will take other portion as validation.

In [37]:
tokenized_datasets_training = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_datasets_validation = tokenized_datasets.filter(lambda example, index: index % 1001 == 0, with_indices=True)

Filter: 100%|██████████| 5161/5161 [00:01<00:00, 3915.40 examples/s]


Check the shapes of all three parts of the dataset:

In [38]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets_training['train'].shape}")
print(f"Validation: {tokenized_datasets_validation['train'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (52, 3)
Validation: (6, 3)
DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'labels'],
        num_rows: 5161
    })
})


The output dataset is ready for fine-tuning.

<a name='2.2'></a>
### 2.2 - Fine-Tune the Model with the Preprocessed Dataset

Now utilize the built-in Hugging Face `Trainer` class (see the documentation [here](https://huggingface.co/docs/transformers/main_classes/trainer)). Pass the preprocessed dataset with reference to the original model. Other training parameters are found experimentally and there is no need to go into details about those at the moment.

In [39]:
output_dir = f'./machine-translation-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets_training['train'],
    eval_dataset=tokenized_datasets_validation['train'] # I pass the same because my dataset doesnt have validation and test
)

Start training process...

In [40]:
trainer.train()

/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.20it/s]

{'loss': 8.4399, 'learning_rate': 0.0, 'epoch': 0.14}
{'train_runtime': 0.465, 'train_samples_per_second': 17.203, 'train_steps_per_second': 2.15, 'train_loss': 8.439949989318848, 'epoch': 0.14}


TrainOutput(global_step=1, training_loss=8.439949989318848, metrics={'train_runtime': 0.465, 'train_samples_per_second': 17.203, 'train_steps_per_second': 2.15, 'train_loss': 8.439949989318848, 'epoch': 0.14})